### Import Packages

In [ ]:
#@title Imports

!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet
!pip install pydot --quiet

     |████████████████████████████████| 24.2 MB 1.2 MB/s 
     |████████████████████████████████| 4.9 MB 8.5 MB/s 
     |████████████████████████████████| 4.4 MB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 


In [ ]:
# Import packages
import pandas as pd
import numpy as np
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from google.colab import drive

from collections import Counter
import matplotlib.pyplot as plt
from nltk.util import ngrams

from transformers import BertTokenizer, TFBertModel
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
import tensorflow as tf


import time
from transformers import create_optimizer

import sklearn as sk
import os
import nltk
from nltk.corpus import reuters
from nltk.data import find

import re

import gensim

In [ ]:
drive.mount('/drive') 
%cd /drive/MyDrive/W266 Project/Colab Notebooks/Exploration
!pwd

Mounted at /drive
/drive/.shortcut-targets-by-id/1p1bDkEjmNKPzX456WZWBr8qtvPr6Pt5m/W266 Project/Colab Notebooks/Exploration
/drive/.shortcut-targets-by-id/1p1bDkEjmNKPzX456WZWBr8qtvPr6Pt5m/W266 Project/Colab Notebooks/Exploration


# Model - BERT

In [ ]:
final_train = pd.read_csv('../../data/transformed/final/train.csv')
final_test = pd.read_csv('../../data/transformed/final/test.csv')

In [ ]:
final_train.head()

,comment_text_transf,violent
0,cocksucker before you piss around on my work,1
1,hey what is it.. @ | talk . what is it an ex...,1
2,"bye don't look, come or think of comming bac...",1
3,you are gay or antisemmitian. archangel whit...,1
4,"fuck your filthy mother in the ass, dry",1


Import helper functions

In [ ]:
#@title Plotting Function

# 4-window plot. Small modification from matplotlib examples.

def make_plot(axs, history1, 
              history2, 
              y_lim_loss_lower=0.4, 
              y_lim_loss_upper=0.6,
              y_lim_accuracy_lower=0.7, 
              y_lim_accuracy_upper=0.8,
              model_1_name='model 1',
              model_2_name='model 2',
              
             ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    ax1 = axs[0, 0]
    ax1.plot(history1.history['loss'])
    ax1.plot(history1.history['val_loss'])
    ax1.set_title('loss - ' + model_1_name)
    ax1.set_ylabel('loss', bbox=box)
    ax1.set_ylim(y_lim_loss_lower, y_lim_loss_upper)

    ax3 = axs[1, 0]
    ax3.set_title('accuracy - ' + model_1_name)
    ax3.plot(history1.history['accuracy'])
    ax3.plot(history1.history['val_accuracy'])
    ax3.set_ylabel('accuracy', bbox=box)
    ax3.set_ylim(y_lim_accuracy_lower, y_lim_accuracy_upper)


    ax2 = axs[0, 1]
    ax2.set_title('loss - ' + model_2_name)
    ax2.plot(history2.history['loss'])
    ax2.plot(history2.history['val_loss'])
    ax2.set_ylim(y_lim_loss_lower, y_lim_loss_upper)

    ax4 = axs[1, 1]
    ax4.set_title('accuracy - ' + model_2_name)

    # small adjustment to account for the 2 accuracy measures in the Weighted Averging Model with Attention
    if 'classification_accuracy' in history2.history.keys():
      ax4.plot(history2.history['classification_accuracy'])
    else:
      ax4.plot(history2.history['accuracy'])
    
    if 'val_classification_accuracy' in history2.history.keys():
      ax4.plot(history2.history['val_classification_accuracy'])
    else:
      ax4.plot(history2.history['val_accuracy'])
    ax4.set_ylim(y_lim_accuracy_lower, y_lim_accuracy_upper)

In [ ]:
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased') # a quick test

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
max_length = 128

x_train = bert_tokenizer(list(final_train['comment_text_transf']),
                         max_length = max_length,
                         truncation = True,
                         padding = 'max_length',
                         return_tensors = 'tf')
y_train = list(final_train['violent'])

In [ ]:
x_test = bert_tokenizer(list(final_test['comment_text_transf'].apply(str)),
                         max_length = max_length,
                         truncation = True,
                         padding = 'max_length',
                         return_tensors = 'tf')
y_test = list(final_test['violent'])

In [ ]:
def create_bert_cls_model(hidden_size = 100, dropout = .3, learning_rate = .00005):
    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    # Build the input layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    # token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    # attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    # Dictionary of inputs
    # bert_inputs = {'input_ids': input_ids,
    #                'token_type_ids': token_type_ids,
    #                'attention_mask': attention_mask} 

    bert_inputs = {'input_ids': input_ids} 

    # model output
    bert_out = bert_model(bert_inputs) 

    # Instead of pooled token, using the 'cls' token 
    # pooled_token = bert_out[1]
    cls_token = bert_out[0][:,0] # First layer, 0th column of each token set


    # Hidden layers
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    # instantiate model
    # classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    classification_model = tf.keras.Model(inputs=input_ids, outputs=[classification])
    
    # compile model
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                            metrics='accuracy')
   
    ### END YOUR CODE
    
    return classification_model

In [ ]:
cls_model = create_bert_cls_model()



cls_model_output = cls_model.fit(x_train, 
                                np.array(y_train),   
                                validation_data=(x_test, np.array(y_test)),    
                                batch_size=8, 
                                epochs=2) 


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/2


ValueError: ignored

In [ ]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

3 159571 3 63978


In [ ]:
x_train[0]

KeyError: ignored

In [ ]:
can you see this

SyntaxError: ignored